In [1]:
%cd ../

/mnt/kireev/pycharm-deploy/vtb


In [2]:
from glob import glob

In [3]:
import numpy as np
import pandas as pd

In [4]:
import torch

In [5]:
import pytorch_lightning as pl

In [6]:
from dltranz.data_preprocessing.pandas_preprocessor import PandasDataPreprocessor

In [7]:
from pyhocon import ConfigFactory

In [8]:
FOLD_ID = 1

In [9]:
fold_id_test = FOLD_ID

In [10]:
folds_count = len(glob('data/train_matching_*.csv'))
folds_count

6

In [82]:
# fold_id_valid = np.random.choice([i for i in range(folds_count) if i != fold_id_test], size=1)[0]
fold_id_valid = (fold_id_test + 1) % folds_count
fold_id_valid

2

In [12]:
# df_matching_train = pd.concat([pd.read_csv(f'data/train_matching_{i}.csv')
#                               for i in range(folds_count) 
#                               if i not in (fold_id_test, fold_id_valid)])
df_matching_valid = pd.read_csv(f'data/train_matching_{fold_id_valid}.csv')
df_matching_test = pd.read_csv(f'data/train_matching_{fold_id_test}.csv')

In [13]:
[len(df) for df in [df_matching_valid, df_matching_test]]

[2930, 2930]

In [14]:
def trx_types(df):
    df['mcc_code'] = df['mcc_code'].astype(str)
    df['currency_rk'] = df['currency_rk'].astype(str)
    df['event_time'] = pd.to_datetime(df['transaction_dttm']).astype(int) / 1e9
    return df[['user_id', 'event_time', 'mcc_code', 'currency_rk', 'transaction_amt']]

In [83]:
# df_trx_train = pd.concat([trx_types(pd.read_csv(f'data/transactions_{i}.csv'))
#                               for i in range(folds_count) 
#                               if i not in (fold_id_test, fold_id_valid)])
df_trx_valid = trx_types(pd.read_csv(f'data/transactions_{fold_id_valid}.csv'))
# df_trx_test = trx_types(pd.read_csv(f'data/transactions_{fold_id_test}.csv'))

In [16]:
def click_types(df):
    df['event_time'] = pd.to_datetime(df['timestamp']).astype(int) / 1e9
    df = pd.merge(df, pd.read_csv('data/click_categories.csv'), on='cat_id')
    df['cat_id'] = df['cat_id'].astype(str)
    return df[['user_id', 'event_time', 'cat_id', 'level_0', 'level_1', 'level_2']]

In [84]:
# df_click_train = pd.concat([click_types(pd.read_csv(f'data/clickstream_{i}.csv'))
#                               for i in range(folds_count) 
#                               if i not in (fold_id_test, fold_id_valid)])
df_click_valid = click_types(pd.read_csv(f'data/clickstream_{fold_id_valid}.csv'))
# df_click_test = click_types(pd.read_csv(f'data/clickstream_{fold_id_test}.csv'))

In [18]:
import pickle
with open('preprocessor_trx.p', 'rb') as f:
    preprocessor_trx = pickle.load(f)
with open('preprocessor_click.p', 'rb') as f:
    preprocessor_click = pickle.load(f)

In [68]:
from dltranz.data_load.iterable_processing.category_size_clip import CategorySizeClip

In [69]:
category_max_size_trx = {
    'mcc_code': 350,
    'currency_rk': 5,
}
category_max_size_click = {
    'cat_id': 400,
    'level_0': 400,
    'level_1': 400,
    'level_2': 400,
}

In [70]:
def trx_to_torch(seq):
    seq = CategorySizeClip(category_max_size_trx)(seq)
    for x in seq:
        yield x['user_id'], {
            'event_time': torch.from_numpy(x['event_time']).float(),
            'mcc_code': torch.from_numpy(x['mcc_code']).int(),
            'currency_rk': torch.from_numpy(x['currency_rk']).int(),
            'transaction_amt': torch.from_numpy(x['transaction_amt']).float(),
        }

def click_to_torch(seq):
    seq = CategorySizeClip(category_max_size_click)(seq)
    for x in seq:
        yield x['user_id'], {
            'event_time': torch.from_numpy(x['event_time']).float(),
            'cat_id': torch.from_numpy(x['cat_id']).int(),
            'level_0': torch.from_numpy(x['level_0']).int(),
            'level_1': torch.from_numpy(x['level_1']).int(),
            'level_2': torch.from_numpy(x['level_2']).int(),

        }

In [71]:
df_click_valid[lambda x: x['user_id'].eq('958c5752b04342c796e38d523816760c')]['cat_id'].value_counts()

Series([], Name: cat_id, dtype: int64)

In [72]:
def new_transform(self, df, copy=True):
    """Perform preprocessing.
    Parameters
    ----------
    df : pandas.DataFrame with flat data
    copy : bool, default=None
        Copy the input X or not.
    Returns
    -------
    features : List of dicts grouped by col_id.
    """
        self.check_is_fitted()
        df_data = df.copy() if copy else df

        # event_time mapping
        if self.time_transformation == 'none':
            pass
        elif self.time_transformation == 'default':
            df_data = self._td_default(df_data, self.cols_event_time)
        elif self.time_transformation == 'float':
            df_data = self._td_float(df_data, self.cols_event_time)
        elif self.time_transformation == 'gender':
            df_data = self._td_gender(df_data, self.cols_event_time)
        else:
            raise NotImplementedError(f'Unknown type of data transformation: "{self.time_transformation}"')

        for col in self.cols_category:
            if col not in self.cols_category_mapping:
                raise KeyError(f"column {col} isn't in fitted category columns")
            pd_col = df_data[col].astype(str)
            df_data[col] = pd_col.map(self.cols_category_mapping[col]) \
                .fillna(max(self.cols_category_mapping[col].values()))

        for col in self.cols_log_norm:
            df_data[col] = np.log1p(abs(df_data[col])) * np.sign(df_data[col])
            df_data[col] /= abs(df_data[col]).max()

        if self.print_dataset_info:
            df = df_data.groupby(self.col_id)['event_time'].count()

        # column filter
        used_columns = [col for col in df_data.columns
                        if col in self.cols_category + self.cols_log_norm + ['event_time', self.col_id]]

        features = df_data[used_columns] \
            .assign(et_index=lambda x: x['event_time']) \
            .set_index([self.col_id, 'et_index']).sort_index() \
            .groupby(self.col_id).apply(lambda x: {k: np.array(v) for k, v in x.to_dict(orient='list').items()}) \
            .rename('feature_arrays').reset_index().to_dict(orient='records')

        def squeeze(rec):
            return {self.col_id: rec[self.col_id], **rec['feature_arrays']}
        features = [squeeze(r) for r in features]


        return features


import types
preprocessor_trx.transform = types.MethodType(new_transform, preprocessor_trx)
preprocessor_click.transform = types.MethodType(new_transform, preprocessor_click)

In [85]:
# features_trx_train = dict(trx_to_torch(preprocessor_trx.fit_transform(df_trx_train)))
features_trx_valid = dict(trx_to_torch(preprocessor_trx.transform(df_trx_valid)))
# features_trx_test = dict(trx_to_torch(preprocessor_trx.transform(df_trx_test)))

In [86]:
# features_click_train = dict(click_to_torch(preprocessor_click.fit_transform(df_click_train)))
features_click_valid = dict(click_to_torch(preprocessor_click.transform(df_click_valid)))
# features_click_test = dict(click_to_torch(preprocessor_click.transform(df_click_test)))

In [26]:
from vtb_code.data import PairedDataset, CrossDataset, paired_collate_fn

In [27]:
from dltranz.data_load import augmentation_chain
from dltranz.data_load.augmentations.seq_len_limit import SeqLenLimit
from dltranz.data_load.augmentations.random_slice import RandomSlice

In [28]:
import random

class RandomSample:
    def __init__(self, min_len, max_len, rate_for_min=1.0):
        super().__init__()

        self.min_len = min_len
        self.max_len = max_len
        self.rate_for_min = rate_for_min

    def __call__(self, x):
        seq_len = len(next(iter(x.values())))

        idx = self.get_idx(seq_len)
        new_x = {k: v[idx] for k, v in x.items()}
        return new_x

    def get_idx(self, seq_len):
        new_idx = np.arange(seq_len)

        min_len, max_len = self.get_min_max(seq_len)
        if max_len < min_len:
            return new_idx
        new_len = random.randint(min_len, max_len)

        return np.sort(np.random.choice(new_idx, size=new_len, replace=False))

    def get_min_max(self, seq_len):
        max_len = int(min(self.max_len, seq_len))
        min_len = int(min(self.min_len, seq_len * self.rate_for_min))
        if min_len < 1:
            min_len = 1
        return min_len, max_len


In [87]:
valid_dataset0 = CrossDataset(
    [
        df_matching_valid['bank'].unique(),
    ], 
    data=[
        features_trx_valid,
    ],
    augmentations=[
        augmentation_chain(SeqLenLimit(2000)),
    ],
)
valid_dataset1 = CrossDataset(
    [
        df_matching_valid['rtk'][lambda x: x.ne('0')].unique(),
    ], 
    data=[
        features_click_valid,
    ],
    augmentations=[
        augmentation_chain(SeqLenLimit(5000)),
    ],
)

In [88]:
valid_dl0 = torch.utils.data.DataLoader(
    valid_dataset0,
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=0,
    batch_size=512,
)
valid_dl1 = torch.utils.data.DataLoader(
    valid_dataset1,
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=0,
    batch_size=512,
)

In [8]:
from dltranz.seq_encoder import create_encoder
from dltranz.metric_learn.sampling_strategies import get_sampling_strategy
from dltranz.metric_learn.losses import get_loss

In [9]:
from vtb_code.metrics import PrecisionK, MeanReciprocalRankK

In [12]:
class PairedModule(pl.LightningModule):
    def __init__(self, params, sampling_strategy_params, loss_params, k,
                 lr, weight_decay,
                 step_size, gamma,
                 base_lr, max_lr, step_size_up, step_size_down,
                ):
        super().__init__()
        self.save_hyperparameters()
        
        self.seq_encoder_trx = torch.nn.Sequential(
            create_encoder(params['trx_seq'], is_reduce_sequence=True),
#             torch.nn.Linear(params['trx_seq.rnn.hidden_size'], params['head_size']),
        )
        self.seq_encoder_click = torch.nn.Sequential(
            create_encoder(params['click_seq'], is_reduce_sequence=True),
#             torch.nn.Linear(params['click_seq.rnn.hidden_size'], params['head_size']),
        )
              
        sampling_strategy = get_sampling_strategy(sampling_strategy_params)
        self.loss_fn = get_loss(loss_params, sampling_strategy)
        
        self.train_precision = PrecisionK(k=k, compute_on_step=False)
        self.train_mrr = MeanReciprocalRankK(k=k, compute_on_step=False)
        self.valid_precision = PrecisionK(k=k, compute_on_step=False)
        self.valid_mrr = MeanReciprocalRankK(k=k, compute_on_step=False)
        
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        if self.hparams.step_size is not None:
            scheduler = torch.optim.lr_scheduler.StepLR(
                optim, step_size=self.hparams.step_size, gamma=self.hparams.gamma)
        else:
            sheduler = torch.optim.lr_scheduler.CyclicLR(
                optim,
                base_lr=self.hparams.base_lr, max_lr=self.hparams.max_lr,
                step_size_up=self.hparams.step_size_up,
                step_size_down=self.hparams.step_size_down,
                cycle_momentum=False,
            )
            scheduler = {'scheduler': sheduler, 'interval': 'step'}
        return [optim], [scheduler]
    
#     def forward(self, batch):
#         x_trx, x_click = batch
#         z_trx = self.seq_encoder_trx(x_trx)  # B, H
#         z_click = self.seq_encoder_click(x_click)  # B, H
        
#         B, H = z_trx.size()
#         logits = (z_trx * z_click).sum(dim=1)  # B(trx),
#         return logits
    
    def pos_pairs(self, a, b):
        l2 = torch.nn.functional.pairwise_distance(a, b)  # B(trx),
        return l2
    
    def neg_pairs(self, a, b):
        l2 = torch.nn.functional.pairwise_distance(a.unsqueeze(1), b.unsqueeze(0))  # B(trx) * B(click)
        B, _ = l2.size()
        device = l2.device
        neg_mask = torch.eye(B, device=device) == 1
        l2 = l2.masked_fill(neg_mask, 10)
        k = min(self.hparams.neg_count, B - 1)
        neg_pairs = torch.topk(l2, k=k, dim=1, largest=False).values
        return neg_pairs
        
    def training_step(self, batch, batch_idx):
        (x_trx, l_trx), (x_click, l_click) = batch
        z_trx = self.seq_encoder_trx(x_trx)  # B, H
        z_click = self.seq_encoder_click(x_click)  # B, H

        z_trx = torch.nn.functional.normalize(z_trx, dim=1)
        z_click = torch.nn.functional.normalize(z_click, dim=1)

        with torch.no_grad():
            n_samples = z_trx.size(0) // (l_trx.max() + 1)
            for i in range(n_samples):
                z_t = z_trx[i::n_samples]
                z_c = z_click[i::n_samples]
                l2 = torch.nn.functional.pairwise_distance(z_t.unsqueeze(1), z_c.unsqueeze(0))
                self.train_precision(-l2)
                self.train_mrr(-l2)
        
        z_common = torch.cat([z_trx, z_click], dim=0)
        l_common = torch.cat([l_trx, l_click], dim=0)
        
        loss = self.loss_fn(z_common, l_common)
        
        return loss

    def validation_step(self, batch, batch_idx):
        (x_trx, l_trx), (x_click, l_click) = batch
        z_trx = self.seq_encoder_trx(x_trx)  # B, H
        z_click = self.seq_encoder_click(x_click)  # B, H

        z_trx = torch.nn.functional.normalize(z_trx, dim=1)
        z_click = torch.nn.functional.normalize(z_click, dim=1)

        with torch.no_grad():
            n_samples = z_trx.size(0) // (l_trx.max() + 1)
            for i in range(n_samples):
                z_t = z_trx[i::n_samples]
                z_c = z_click[i::n_samples]
                l2 = torch.nn.functional.pairwise_distance(z_t.unsqueeze(1), z_c.unsqueeze(0))
                self.valid_precision(-l2)
                self.valid_mrr(-l2)

    def training_epoch_end(self, _):
        self.log('train_metrics/precision', self.train_precision, prog_bar=True)
        self.log('train_metrics/mrr', self.train_mrr, prog_bar=False)

    def validation_epoch_end(self, _):
        self.log('valid_metrics/precision', self.valid_precision, prog_bar=True)
        self.log('valid_metrics/mrr', self.valid_mrr, prog_bar=False)
    

In [14]:
PairedModule.load_from_checkpoint('lightning_logs/version_40/checkpoints/epoch=49-step=999.ckpt').hparams

"base_lr":                  0.0005
"gamma":                    0.3
"k":                        17
"loss_params":              ConfigTree([('train', ConfigTree([('loss', 'HistogramLoss')]))])
"lr":                       0.004
"max_lr":                   0.004
"params":                   ConfigTree([('trx_seq', ConfigTree([('trx_encoder', ConfigTree([('use_batch_norm_with_lens', False), ('norm_embeddings', False), ('embeddings_noise', 0.0), ('embeddings', ConfigTree([('mcc_code', ConfigTree([('in', 350), ('out', 32)])), ('currency_rk', ConfigTree([('in', 10), ('out', 4)]))])), ('numeric_values', ConfigTree([('transaction_amt', 'identity')]))])), ('encoder_type', 'rnn'), ('rnn', ConfigTree([('type', 'gru'), ('hidden_size', 32), ('bidir', False), ('trainable_starter', 'static')]))])), ('click_seq', ConfigTree([('trx_encoder', ConfigTree([('use_batch_norm_with_lens', False), ('norm_embeddings', False), ('embeddings_noise', 0.0), ('embeddings', ConfigTree([('cat_id', ConfigTree([('in', 400),

In [17]:
PairedModule.load_from_checkpoint('lightning_logs/version_37/checkpoints/epoch=49-step=999.ckpt').hparams['params']

ConfigTree([('trx_seq',
             ConfigTree([('trx_encoder',
                          ConfigTree([('use_batch_norm_with_lens', False),
                                      ('norm_embeddings', False),
                                      ('embeddings_noise', 0.0),
                                      ('embeddings',
                                       ConfigTree([('mcc_code',
                                                    ConfigTree([('in', 350),
                                                                ('out', 32)])),
                                                   ('currency_rk',
                                                    ConfigTree([('in', 10),
                                                                ('out',
                                                                 4)]))])),
                                      ('numeric_values',
                                       ConfigTree([('transaction_amt',
                                     

In [34]:
params =     ConfigFactory.parse_string('''
    trx_seq: {
        trx_encoder: {
          use_batch_norm_with_lens: false
          norm_embeddings: false,
          embeddings_noise: 0.000,
          embeddings: {
            mcc_code: {in: 350, out: 64},
            currency_rk: {in: 10, out: 4}
          },
          numeric_values: {
            transaction_amt: identity
          }
        },
        encoder_type: rnn,
        rnn: {
          type: gru,
          hidden_size: 128,
          bidir: false,
          trainable_starter: static
        }
    }
    click_seq: {
        trx_encoder: {
          use_batch_norm_with_lens: false
          norm_embeddings: false,
          embeddings_noise: 0.000,
          embeddings: {
            cat_id: {in: 400, out: 64},
            # level_0: {in: 400, out: 16}
            # level_1: {in: 400, out: 8}
            # level_2: {in: 400, out: 4}
          },
          numeric_values: {
          }
        },
        encoder_type: rnn,
        rnn: {
          type: gru,
          hidden_size: 128,
          bidir: false,
          trainable_starter: static
        }    
    }
''')

In [77]:
sup_model = PairedModule.load_from_checkpoint('lightning_logs/version_15/checkpoints/epoch=2999-step=29999.ckpt',
                                              params=params)

In [78]:
device = torch.device('cuda:1')

In [79]:
sup_model.eval()
sup_model.to(device);

In [80]:
from tqdm.auto import tqdm

In [89]:
m_valid0 = []
with torch.no_grad():
    for batch in tqdm(valid_dl0):
        z_trx = sup_model.seq_encoder_trx(batch[0].to(device))  # B, H
        z_trx = torch.nn.functional.normalize(z_trx, dim=1)
        m_valid0.append(z_trx.cpu())


m_valid1 = []
with torch.no_grad():
    for batch in tqdm(valid_dl1):
        z_click = sup_model.seq_encoder_click(batch[0].to(device))  # B, H
        z_click = torch.nn.functional.normalize(z_click, dim=1)
        m_valid1.append(z_click.cpu())


m_valid = pd.DataFrame(
    torch.nn.functional.pairwise_distance(
        torch.cat(m_valid0, axis=0).unsqueeze(1),
        torch.cat(m_valid1, axis=0).unsqueeze(0),
    ).numpy(), 
    index=valid_dataset0.unique_ids[0],
    columns=valid_dataset1.unique_ids[0],
)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [90]:
pre = 0.0
mrr = 0.0

for ix_bank, s_scores in tqdm(m_valid.iterrows()):
    d = s_scores.sort_values().iloc[:100].rename('l2').reset_index().rename(columns={'index': 'rtk'}) \
    .assign(rank=1 / (np.arange(100) + 1))[lambda x: x['rtk'].eq(
        df_matching_valid.set_index('bank').loc[ix_bank, 'rtk'])]
    if len(d) == 1:
        pre += 1
        mrr += d['rank'].iloc[0]
pre = pre / len(m_valid)
mrr = mrr / len(m_valid)
r1 = 2 * pre * mrr / (pre + mrr)

0it [00:00, ?it/s]

In [91]:
print(f'pre: {pre:.3f}, mrr: {mrr:.3f}, r1: {r1:.3f}')

pre: 0.078, mrr: 0.005, r1: 0.010


In [105]:
pre = 0.0
mrr = 0.0

for ix_bank, s_scores in tqdm(
    m_valid.iloc[:, np.linspace(0, m_valid.shape[1] - 1, 1024).astype(int)].iterrows()):
    d = s_scores.sort_values().iloc[:34].rename('l2').reset_index().rename(columns={'index': 'rtk'}) \
    .assign(rank=1 / (np.arange(34) + 1))[lambda x: x['rtk'].eq(
        df_matching_valid.set_index('bank').loc[ix_bank, 'rtk'])]
    if len(d) == 1:
        pre += 1
        mrr += d['rank'].iloc[0]
pre = pre / len(m_valid)
mrr = mrr / len(m_valid)
r1 = 2 * pre * mrr / (pre + mrr)

0it [00:00, ?it/s]

In [106]:
print(f'pre: {pre:.3f}, mrr: {mrr:.3f}, r1: {r1:.3f}')

pre: 0.025, mrr: 0.003, r1: 0.006


In [101]:
100 / 3000 * 1024

34.13333333333333

In [55]:
print(f'pre: {pre:.3f}, mrr: {mrr:.3f}, r1: {r1:.3f}')

pre: 0.051, mrr: 0.003, r1: 0.005
